# Скрипт предсказывающий площадь

## Загрузим и обработаем тестовые данные

In [2]:
import pandas as pd
import joblib
import numpy as np

data = pd.read_csv('test_data.csv')

### Парсинг эмбеддингов

In [3]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

In [4]:
new_embedding = []
for str in data['embedding']:
    result = []
    if '\n' in str:
        res = str.split('\n')
    else:
        res = str.split('], ')
    for mass in res:
        s = mass.replace(', dtype=float32',' ').replace('(', ' ').replace(')',' ').replace(',',' ').replace('[','').replace(']','').split(' ')
        tmp = []
        for word in s:
            if is_number(word):
                tmp.append(float(word))
        result.append(tmp)
    r = [np.array(sublist) for sublist in result]
    new_embedding.append(r)

### Дополняем эмбеддинги до одной размерности

In [5]:
import pickle

# Загрузка значения из файла
file_path = 'max_subarrays.pickle'
with open(file_path, 'rb') as f:
    max_subarrays = pickle.load(f)

In [6]:
for i in range(len(new_embedding)):
    while len(new_embedding[i]) < max_subarrays:
        new_embedding[i] = np.vstack([new_embedding[i], np.zeros(25)])

### Запишем все имеющиеся в представлении вектора друг за другом, по порядку, по которому они хранятся изначально

In [7]:
for i in range(len(new_embedding)):
    tmp = []
    for j in range (len(new_embedding[i])):
        for k in new_embedding[i][j]:
            tmp.append(k)
    new_embedding[i] = np.array(tmp)

In [8]:
s = pd.Series(new_embedding)
data['New_embeddings'] = s

### Преобразуем данные в формат таблицы

In [9]:
X = pd.DataFrame(data['New_embeddings'].values.tolist(), index = data['New_embeddings'].index)

## Загружаем обученную модель и предсказываем значения

### Площадь

In [10]:
# Шаг 2: Загрузка обученной модели
model_area = joblib.load('trained_model_area.pkl') 

# Шаг 3: Применение модели к данным и получение предсказаний
predictions_area = model_area.predict(X)  

### Задержка

In [ ]:
# Шаг 2: Загрузка обученной модели
model_delay = joblib.load('trained_model_delay.pkl')

# Шаг 3: Применение модели к данным и получение предсказаний
predictions_delay = model_delay.predict(X)

### Создание и выгрузка таблицы

In [12]:
result = pd.DataFrame()

In [ ]:
result['id'] = data['File']
result['Area'] = predictions_area
result['Delay'] = predictions_delay

In [14]:
result.to_csv('submission.csv', index=False)